based on: https://www.kaggle.com/code/raufmomin/vision-transformer-vit-fine-tuning

In [1]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from img_utils import *
from multiprocessing import Process, Queue
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/tmp/ipykernel_41341/485958369.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-03-11 10:28:57.946282: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 10:28:57.946340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 10:28:57.947181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable t

./kaggle/input/vit-b32-classification/saved_model.pb
./kaggle/input/vit-b32-classification/variables/variables.data-00000-of-00001
./kaggle/input/vit-b32-classification/variables/variables.index
./kaggle/input/vit-b32-fe/saved_model.pb
./kaggle/input/vit-b32-fe/variables/variables.data-00000-of-00001
./kaggle/input/vit-b32-fe/variables/variables.index
./kaggle/input/vit-b8-fe/saved_model.pb
./kaggle/input/vit-b8-fe/variables/variables.data-00000-of-00001
./kaggle/input/vit-b8-fe/variables/variables.index


In [2]:
# Select an Image Classification model
model_name = "vit-b32-classification"

model_handle_map = {
  "vit-b8-fe": "./kaggle/input/vit-b8-fe",
  "vit-b32-fe": "./kaggle/input/vit-b32-fe",
  "vit-b32-classification": "./kaggle/input/vit-b32-classification",
}


model_image_size_map = {
  "vit-b8-fe": 224,
  "vit-b32-fe": 224,
  "vit-b32-classification": 224,
}

model_handle = model_handle_map.get(model_name)
pixels = model_image_size_map.get(model_name, 224)

print(f"Selected model: {model_name} : {model_handle}")

IMAGE_SIZE = (pixels, pixels)
print(f"Image size: {IMAGE_SIZE}")

BATCH_SIZE = 1

Selected model: vit-b32-classification : ./kaggle/input/vit-b32-classification
Image size: (224, 224)


In [3]:
def build_dataset(subset):
    data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

    return tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=.20,
        subset=subset,
        label_mode="categorical",
        # Seed needs to provided when using validation_split and shuffle = True.
        # A fixed seed is used so that the validation set is stable across runs.
        seed=123,
        image_size=IMAGE_SIZE,
        batch_size=1)

def parse_data(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_path = os.path.join(root, file)
                with open(json_path) as f:
                    data = json.load(f)
                    price = data['price']
                    image_path = os.path.join(root, '0.jpg')
                    # Get image as tensor
                    try:
                        image = tf.io.read_file(image_path)
                        image = tf.image.decode_jpeg(image, channels=3)
                        # Resize image
                        image = tf.image.resize(image, IMAGE_SIZE)
                        # Normalize image
                        image = image/255.0
                        yield price, image
                    except Exception as e:
                        print(f"Error reading image {image_path}: {e}")
                        continue

def preprocess_image(image_path):
    pass

def build_custom_dataset(directory: str, preprocces_data: bool = False, validation_split: float = 0.2) -> tf.data.Dataset:
    data_generator = lambda: parse_data(directory)
    dataset = tf.data.Dataset.from_generator(
        data_generator,
        output_signature=(tf.TensorSpec(shape=(), dtype=tf.float32), tf.TensorSpec(shape=(pixels, pixels, 3), dtype=tf.float32))
    )
    print(f"Found {len(list(dataset))} images in {directory}")
    dataset = dataset.map(lambda price, image: (image, price))
    # Reshape dataset
    dataset = dataset.batch(BATCH_SIZE)
    # Split dataset
    train_size = int((1-validation_split) * len(list(dataset)))
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size)
    return train_dataset, val_dataset

    

In [4]:
# Combine and train the model on custom regression data, with no labels
def compile_and_fit_regression(epochs=50, do_fine_tuning=False):  # TODO: fuck this, try manually like in cnn_regression.ipynb
  set_gpu()

  print("Getting training data")
  train_ds, valid_ds = build_custom_dataset("../nybolig-scrape/output")
  train_size = len(list(train_ds))
  valid_size = len(list(valid_ds))
  
  # Show some images
  plt.figure(figsize=(10, 10))
  for images, labels in train_ds.take(1):
    for i in range(9):
      ax = plt.subplot(3, 3, i + 1)
      print(images)
      plt.imshow(images.numpy().astype("uint8"))
      plt.title(labels.numpy())
      plt.axis("off")

  print("Building model with", model_handle)
  model = tf.keras.Sequential([
      # Explicitly define the input shape so the model can be properly
      # loaded by the TFLiteConverter
      tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
      hub.KerasLayer(model_handle, trainable=do_fine_tuning),
      tf.keras.layers.Dropout(rate=0.2),
      tf.keras.layers.Dense(1)
  ])
  model.build((None,)+IMAGE_SIZE+(3,))
  model.summary()

  print("Compiling model")
  model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mae'])
  
  print("Training model")
  steps_per_epoch = train_size // BATCH_SIZE
  validation_steps = valid_size // BATCH_SIZE
  print(f"SPE: {steps_per_epoch}, VS: {validation_steps}")
  hist = model.fit(
      train_ds,
      epochs=epochs, steps_per_epoch=steps_per_epoch,
      validation_data=valid_ds,
      validation_steps=validation_steps).history
  
  print("Final training loss", hist["loss"][-1])
  print(hist)

  # Visualize the training and validation accuracy
  plt.figure()
  plt.ylabel("Mean Abs Error (training and validation)")
  plt.xlabel("Training Steps")
  plt.ylim([0,2])
  plt.plot(hist["mae"])
  plt.plot(hist["val_mae"])
  plt.legend(loc='lower right')

  # Visualize the training and validation loss
  plt.figure()
  plt.ylabel("Loss (training and validation)")
  plt.xlabel("Training Steps")
  plt.ylim([0,2])
  plt.plot(hist["loss"])
  plt.plot(hist["val_loss"])
  plt.legend(loc='upper right')

  plt.show()

  # Try predictions
  price_batch = np.array([price for price, image in valid_ds])
  image_batch = np.array([image for price, image in valid_ds])
  predicted_batch = model.predict(image_batch)
  plt.figure(figsize=(10,9))
  plt.scatter(price_batch, predicted_batch)
  plt.xlabel('True Values [Price]')
  plt.ylabel('Predictions [Price]')
  plt.axis('equal')
  plt.axis('square')
  plt.xlim([0,plt.xlim()[1]])
  plt.ylim([0,plt.ylim()[1]])
  _ = plt.plot([-100, 100], [-100, 100])
  plt.show()
  
p = Process(target=compile_and_fit_regression, args=(2, False))
# p = Process(target=compile_and_fit_classification, args=(10, False))
p.start()
p.join()

2024-03-11 10:29:00.770642: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 10:29:00.796424: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 10:29:00.796523: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 10:29:00.799664: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 10:29:00.799772: I external/local_xla/xla/stream_executor

1 Physical GPUs, 1 Logical GPU
Getting training data


2024-03-11 10:29:00.986818: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 10:29:00.986985: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 10:29:00.987004: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2022] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-03-11 10:29:00.987063: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 10:29:00.987086: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created

Found 735 images in ../nybolig-scrape/output


KeyboardInterrupt: 

tf.Tensor(
[[[[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  ...

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]]], shape=(1, 224, 224, 3), dtype=float32)


Process Process-1:
Traceback (most recent call last):
  File "/home/linuxbrew/.linuxbrew/opt/python@3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/linuxbrew/.linuxbrew/opt/python@3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_41341/2548349563.py", line 16, in compile_and_fit_regression
    plt.imshow(images.numpy().astype("uint8"))
  File "/home/besplago/.pyenv/versions/penv/lib/python3.11/site-packages/matplotlib/pyplot.py", line 3343, in imshow
    __ret = gca().imshow(
            ^^^^^^^^^^^^^
  File "/home/besplago/.pyenv/versions/penv/lib/python3.11/site-packages/matplotlib/__init__.py", line 1465, in inner
    return func(ax, *map(sanitize_sequence, args), **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/besplago/.pyenv/versions/penv/lib/python3.11/site-packages/matplotlib/axes/_axes.py", line 5756, in 